In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction :

Dans ce notebook, nous exploitons les capacités d’un **modèle de langage open source (LLM)** — ici **Gemma 3B**, un modèle compatible avec `llama.cpp` — pour effectuer de la génération de texte et de code **en local sur Kaggle**, sans avoir recours à une API externe.

Voici ce que ce projet nous permettra d’expérimenter concrètement :

-  **Chargement d’un LLM optimisé** au format GGUF (Gemma 3B) à l’aide de `llama-cpp-python`, une interface Python pour `llama.cpp`.
-  **Initialisation manuelle du modèle** avec contrôle sur les paramètres (`n_ctx`, `n_threads`, `n_gpu_layers`), ce qui vous permet de comprendre l’impact des ressources matérielles.
-  **Génération de texte à partir de prompts naturels**, pour explorer la capacité du modèle à produire des réponses cohérentes et informatives.
-  **Génération de code Python** à partir d’instructions simples, testant les compétences du modèle dans des tâches proches de Copilot ou ChatGPT.
-  **Streaming token par token**, pour simuler un affichage progressif des réponses comme dans une interface de chatbot.
-  **Exercices pratiques** visant à tester les compétences du modèle sur des cas concrets : fonctions, scripts de scraping, explications en langage naturel, etc.

1. Install llama-cpp-python

In [2]:
# Install llama-cpp-python quietly
!pip install llama-cpp-python > /dev/null 2>&1
print(" llama-cpp-python installed successfully.")


 llama-cpp-python installed successfully.


2. Download a compatible model in GGUF format (Gemma 3B)

In [3]:
# Download GGUF model (Gemma 3B, Q8_0 quantized)
!wget -q https://huggingface.co/MaziyarPanahi/gemma-3-1b-it-GGUF/resolve/main/gemma-3-1b-it.Q8_0.gguf -O gemma-3-1b-it.Q8_0.gguf
print(" Model downloaded successfully (gemma-3-1b-it.Q8_0.gguf)")


 Model downloaded successfully (gemma-3-1b-it.Q8_0.gguf)


3. Load the model with llama_cpp
python
Copier le code


In [5]:
from llama_cpp import Llama

llm = Llama(
    model_path="gemma-3-1b-it.Q8_0.gguf",
    n_ctx=2048,
    n_threads=8,       # Adjust based on CPU
    n_gpu_layers=0,    # Set to 0 if no GPU
    verbose=False
)

print("Model loaded successfully")


llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


Model loaded successfully


4. Quick prompt test (text generation)

In [6]:
prompt = " What is the difference between a star and a planet?"
output = llm(prompt, max_tokens=100)
print(" Response:", output["choices"][0]["text"])


 Response: 

A star is a massive, luminous ball of plasma that produces its own light and heat through nuclear fusion. Planets are celestial bodies that orbit a star and are large enough to have cleared their orbital path.

Here's a breakdown of the key differences:

*   **Light and Heat:** Stars generate their own light and heat through nuclear fusion, while planets reflect light from a star.
*   **Orbit:** Stars orbit a central point, while planets orbit stars.
*   **


5. Code generation example

In [7]:
prompt_code = "Write a Python script that loads a model from Hugging Face Transformers."

output = llm(prompt_code, max_tokens=120)
print(" Generated Code:\n", output["choices"][0]["text"])


 Generated Code:
 

```python
from transformers import pipeline

# Load a pre-trained model
model_name = "bert-base-uncased"  # Change this to your desired model
try:
    model = pipeline("text-generation", model=model_name)
    print(f"Model '{model_name}' loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
```

Key improvements and explanations:

* **Error Handling:** The `try...except` block is crucial.  It gracefully handles potential errors


 6. Streamed response (token by token)

In [8]:
prompt_stream = "Write a Python function that returns True if a number is prime."

output_stream = llm(prompt_stream, max_tokens=80, stream=True)

print(" Streaming response:\n")
for chunk in output_stream:
    print(chunk["choices"][0]["text"], end="", flush=True)


 Streaming response:



```python
def is_prime(n):
  if n <= 1:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      return False
  return True
```

**Explanation:**

1. **`def is_prime(n):

7. Bonus: Try your own prompts

In [9]:
bonus_prompts = [
    "Write a Python function to check if a number is prime.",
    "Explain the difference between a list and a tuple in Python.",
    "Create a script to read a CSV file and plot a line chart using matplotlib.",
    "Generate a web scraper that extracts headlines from a news website using requests and BeautifulSoup.",
    "Write a for loop that prints even numbers between 1 and 100."
]

for i, p in enumerate(bonus_prompts, 1):
    print(f"\n\n--- Prompt {i} ---\n {p}")
    out = llm(p, max_tokens=100)
    print(out["choices"][0]["text"])




--- Prompt 1 ---
 Write a Python function to check if a number is prime.


```python
def is_prime(n):
  """
  Check if a number is prime.

  Args:
    n: The number to check.

  Returns:
    True if the number is prime, False otherwise.
  """
  if n <= 1:
    return False
  for i in range(2, int(n**0.5) + 1):
    if n % i == 0:
      


--- Prompt 2 ---
 Explain the difference between a list and a tuple in Python.


**Python Lists**

*   **Mutable:** Lists are mutable, meaning you can change their contents after they are created. You can add, remove, or modify elements.
*   **Ordered:** Lists are ordered, meaning the elements are stored in a specific sequence.
*   **Heterogeneous:** Lists can contain elements of different data types (e.g., integers, strings, floats).
*   **Syntax:** You define a list using square brackets `[]`.

**


--- Prompt 3 ---
 Create a script to read a CSV file and plot a line chart using matplotlib.


```python
import csv
import matplotlib.pyplot as plt
impo